In [ ]:
import h1st as h1
h1.init()

import pandas as pd
import numpy as np
import sklearn.metrics


In [ ]:
from AutomotiveCybersecurity.models.msg_freq_event_detector import MsgFreqEventDetectorModel
m = MsgFreqEventDetectorModel()
data = m.load_data(num_files=1)

In [ ]:
m.train(data)

In [ ]:
# Don't run automatically this easily overwite latest version in AHT's computer, 
# I need to use correct version in the tutorial notebooks
# m.persist()

In [ ]:
m.stats

In [ ]:
from AutomotiveCybersecurity.graph import WindowGenerator

df = pd.read_parquet(data['attack_files'][0])

graph = h1.Graph()
graph.start()\
     .add(WindowGenerator())\
     .add(MsgFreqEventDetectorModel().load())
graph.end()
results = graph.predict({"df": df})

results["event_detection_results"]

In [ ]:
len(results["event_detection_results"])

In [ ]:
att_wins1 = len([x for x in results["event_detection_results"] if x["WindowInAttack"]])

In [ ]:
from AutomotiveCybersecurity.models.gradient_boosting_msg_classifier import GradientBoostingMsgClassifierModel

In [ ]:
m2 = GradientBoostingMsgClassifierModel()
data = m2.load_data(4)
prepared_data = m2.prep_data(data)

In [ ]:
m2.train(prepared_data)

In [ ]:
# Don't run automatically this easily overwite latest version in AHT's computer, 
# I need to use correct version in the tutorial notebooks
# m2.persist()

In [ ]:
m2.evaluate(prepared_data)

In [ ]:
m2.metrics['confusion_matrix']

In [ ]:
from AutomotiveCybersecurity.graph import WindowGenerator

class NoOp(h1.Action):
    def call(self, command, inputs):
        pass

graph = h1.Graph()
graph.start()\
     .add(WindowGenerator())\
     .add(h1.Decision(m, decision_field="WindowInAttack"))\
     .add(yes=m2,
          no=NoOp())
graph.end()

df = pd.read_csv(data['test_attack_files'][0])
df.columns = ['Timestamp', 'Label', 'CarSpeed', 'SteeringAngle', 'YawRate', 'Gx', 'Gy',]

results = graph.predict({"df": df})

print(sklearn.metrics.confusion_matrix(results['injection_window_results']["Label"] == "Tx", 
                                       results['injection_window_results']["MsgIsAttack"]))
print(sklearn.metrics.accuracy_score(results['injection_window_results']["Label"] == "Tx", 
                                       results['injection_window_results']["MsgIsAttack"]))

In [ ]:
att_wins2 = len([x for x in results["event_detection_results"] if x["WindowInAttack"]])
att_wins2

In [ ]:
assert att_wins2 == att_wins1

In [ ]:
from AutomotiveCybersecurity.util import evaluate_event_graph

evaluate_event_graph(graph, prepared_data['test_files'][-2:])